In [1]:
#IMPORTING MODULES

import numpy as np
import pandas as pd
import os
from astropy.timeseries import LombScargle
import lightkurve as lk
import matplotlib.pyplot as plt
import shutil
from astropy.utils.data import _get_download_cache_loc

from astroquery.simbad import Simbad
from astroquery.gaia import Gaia
from astroquery.mast import Catalogs

In [2]:
#DATA GENERATOR

#getting name
def id_r(tic_id):
    try:
        name = f"TIC {tic_id}"

        result = Simbad.query_objectids(name)

        return result['id'][2]
    except:
        return None

#getting spectral type
def spec_r(tic_id):
    try:
        name = f"TIC {tic_id}"
    
        query = f"""
            SELECT b.sp_type
            FROM basic AS b
            JOIN ident AS i
            ON b.oid = i.oidref
            WHERE i.id = '{name}'"""

        result = Simbad.query_tap(query)

        return result['sp_type'][0]
    
    except:
        return None

#Distance (parsecs, input parallax is in mas)
def dist(tic_id):
    try:
        catalog_data = Catalogs.query_criteria(catalog="Tic", ID=tic_id)
        plx = catalog_data['plx'][0]
        dist = (1/plx)*1e3
        return dist
    
    except:
        return None

#gaia mean mag (apparent)
def apar_mag_r(tic_id):
    try:
        catalog_data = Catalogs.query_criteria(catalog="Tic", ID=tic_id)
        result = catalog_data['GAIAmag'][0]
        return result
        
    except:
        return None

#Absolute gaia Mag
def abs_mag_r(tic_id):
    try:
        catalog_data = Catalogs.query_criteria(catalog="Tic", ID=tic_id)
        apar = catalog_data['GAIAmag'][0]
        plx = catalog_data['plx'][0]
        dist = (1/plx)*1e3
        return apar - 5*np.log10(dist/10)
    except:
        return None

#bp_rp (gaia mag)
def bp_rp_r(tic_id):
    try:
        catalog_data = Catalogs.query_criteria(catalog="Tic", ID=tic_id)
        gaia_bp = catalog_data['gaiabp'][0]
        gaia_rp = catalog_data['gaiarp'][0]
        return gaia_bp - gaia_rp
    except:
        return None

#RA and DEC
def ra_dec(tic_id):
    try:
        catalog_data = Catalogs.query_criteria(catalog="Tic", ID=tic_id)
        ra = catalog_data['ra'][0]
        dec = catalog_data['dec'][0]
        return (ra, dec)
    except:
        return None


#Summary Variables
def sum_gen(tic_id):
    return id_r(tic_id),spec_r(tic_id),dist(tic_id),apar_mag_r(tic_id),abs_mag_r(tic_id),bp_rp_r(tic_id),ra_dec(tic_id)


In [3]:
#FUNDAMENTAL FREQUENCY
def fund_freq(abs_mag):
    period = 10**((abs_mag + 1.4)/(-3.01))

    fund_freq = 1/period

    return fund_freq

In [4]:
#GRAPHICAL ANALYSIS **************

#search function

def selective_search(tic_id):
    search_result = lk.search_lightcurve(f'TIC {tic_id}', mission='TESS')

    if not search_result:
        print(f"⚠️ No TESS light curves found for TIC {tic_id}.")
        return None
    
    try:
        selec_search = search_result[search_result.author == 'SPOC']
        exptimes = selec_search.table['exptime']
        min_index = exptimes.argmin()
        parsed_search = selec_search[min_index]
    except:
        print("No Spoc result found")
        exptimes = search_result.table['exptime']
        min_index = exptimes.argmin()
        parsed_search = search_result[min_index]
        
    return parsed_search

#downloading data and removing bad data points function

def dwnlwd(parsed_search):
    data = parsed_search.download(quality_bitmask='default').remove_nans()
    return data

#Removing Outliers
def outlier_remover(data):
    return data.remove_outliers(5)

#Extracting and normalising data
def extracter(data):
    #.remove_outliers()
    
    time, flux = data.time.value, data.flux.value
    flux /= np.median(flux)
    time -= time[0]

    return time, flux

#lightcurve plotter function

def intensity_plot(time,flux,save, tic_id):
    plt.figure(figsize = (14,8))
    plt.scatter(time, flux, s = 0.5, c='black')
    plt.xlabel("Time Elapsed (days)")
    plt.ylabel("Normalised flux (dimensionless)")
    plt.title(f"Normalised lightcurve for TIC ID:{tic_id}")
    if save == "T":
        plt.savefig(f"lightcurve_of_{tic_id}.png")
    plt.close()

#Fourier Transform function
def calc_lomb_scargle(t,y):
    oversample = 10 # can be adjusted
    tmax = t.max()
    tmin = t.min()
    df = 1.0 / (tmax - tmin)
    fmin = df
    fmax = 1000 # set max freq in c/d

    freq = np.arange(fmin, fmax, df / oversample)
    model = LombScargle(t, y)
    sc = model.power(freq, method="fast", normalization="psd")

    fct = np.sqrt(4./len(t))
    amp = np.sqrt(sc) * fct * 1e6
    return freq, amp # freq in cycles per day and amp in ppm

#fourier transform plotter function
def fourier_plot_1(freq,amp,save, tic_id):
    plt.figure(figsize = (14, 8))
    plt.plot(freq, amp, c ='black')
    plt.xlabel("Frequency (cycles per day)")
    plt.ylabel("Amplitude (ppm)")
    plt.xlim(0,70)
    ff = fund_freq(abs_mag_r(tic_id))
    plt.axvline(ff, color='r', linestyle='--', label='Fundamental Frequency')
    plt.title(f"Fourier Transform for Intensity Oscillations of TIC ID:{tic_id}")
    if save == "T":
        plt.savefig(f"fourier1_of_{tic_id}.png")
    plt.close()

#Close up (if needed)
def fourier_plot_2(freq,amp,save, tic_id):
    plt.figure(figsize = (14, 8))
    plt.plot(freq, amp, c ='black')
    plt.xlabel("Frequency (cycles per day)")
    plt.ylabel("Amplitude (ppm)")
    plt.xlim(0,20)
    ff = fund_freq(abs_mag_r(tic_id))
    plt.axvline(ff, color='r', linestyle='--', label='Fundamental Frequency')
    plt.title(f"Fourier Transform for Intensity Oscillations of TIC ID:{tic_id}")
    if save == "T":
        plt.savefig(f"fourier2_of_{tic_id}.png")
    plt.close()


#Compiling Function

def star_analysis(tic_id,save):

    parsed_search_result = selective_search(tic_id)

    if parsed_search_result == None:
        return
    
    data = dwnlwd(parsed_search_result)

    data_stripped = outlier_remover(data)

    time, flux = extracter(data_stripped)

    intensity_plot(time,flux,save, tic_id)

    freq, amp = calc_lomb_scargle(time, flux)

    fourier_plot_1(freq,amp,save, tic_id)

    fourier_plot_2(freq,amp,save, tic_id)



In [5]:
#PDF GENERATOR***********

from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image

from reportlab.lib.styles import getSampleStyleSheet


#creating information

def info_creator(tic_id):
    star_analysis(tic_id,"T")
    name, spec_type, distance, apparent_mag, absolute_mag, bp_rp, ra_dec = sum_gen(tic_id)
    return name, spec_type, distance, apparent_mag, absolute_mag, bp_rp, ra_dec

#pdf generator function
def pdf_creator(tic_id):
    name, spec_type, distance, apparent_mag, absolute_mag, bp_rp, ra_dec = info_creator(tic_id)

    pdf_name = f"{tic_id} Analysis Sheet"
    doc = SimpleDocTemplate(f"confirmed_scutis/{pdf_name}.pdf", leftMargin=20,rightMargin=20, topMargin=20, bottomMargin=20)

    styles = getSampleStyleSheet()
    story = []

    # Use conditional expressions to format values only if they are not None
    formatted_distance = f"{distance:.2f}" if distance is not None else "N/A"
    formatted_apparent_mag = f"{apparent_mag:.2f}" if apparent_mag is not None else "N/A"
    formatted_absolute_mag = f"{absolute_mag:.2f}" if absolute_mag is not None else "N/A"
    formatted_bp_rp = f"{bp_rp:.2f}" if bp_rp is not None else "N/A"
    formatted_coords = f"{ra_dec[0]}, {ra_dec[1]}" if all(coord is not None for coord in ra_dec) else "N/A"

    text = f"""
<b>Summary on chosen star designated by TICID:</b> {tic_id}<br/>
<b>Common Name:</b> {name}<br/>
<b>Spectral Type:</b> {spec_type}<br/>
<b>Distance:</b> {formatted_distance} (pc)<br/>
<b>Equitorial Coordinates:</b> {formatted_coords} (deg)<br/>
<b>Apparent GAIA Magnitude:</b> {formatted_apparent_mag}<br/>
<b>Absolute GAIA Magnitude:</b> {formatted_absolute_mag}<br/>
<b>BP-RP Value (Gaia):</b> {formatted_bp_rp}
"""
    story.append(Paragraph(text, styles["Normal"]))
    story.append(Spacer(1, 20))

    image_files_to_add = [f"lightcurve_of_{tic_id}.png", f"fourier1_of_{tic_id}.png", f"fourier2_of_{tic_id}.png"]
    
    for fname in image_files_to_add:
        if os.path.exists(fname):
            try:
                img = Image(fname)
                img._restrictSize(doc.width, doc.height)
                story.append(img)
                story.append(Spacer(1, 20))
            except Exception as e:
                print(f"Failed to add image {fname} to PDF: {e}")
    
    doc.build(story)
    
    # After the PDF is built, delete the image files
    for fname in image_files_to_add:
        if os.path.exists(fname):
            os.remove(fname)


In [7]:
###Creating the dscuti list

#Raw table
raw_table = pd.read_csv("tess_sector_91_92.csv",skiprows=[0, 1],names=["update_date","main_id","TICID","gaiadr3_id","CCD","Tmag","RA","Dec","sector","count","sp_type","sp_qual","plx_value","V","B","G","otype","nbref","rvz_radvel","rvz_redshift","gaiadr3_plx","gaiadr3phot_g_mean_mag","gaiadr3_bp_rp","abs_mag_rough"])

#Creating scuti line
point_1 = (0.24,2)
point_2 = (0.12,4)

m = (point_1[1]-point_2[1])/(point_1[0] - point_2[0])
b= (point_1[1]-m*point_1[0])
tolerance = 7

#creating boolean mask
raw_table['gaiadr3_bp_rp'] = pd.to_numeric(raw_table['gaiadr3_bp_rp'], errors='coerce')
raw_table['abs_mag_rough'] = pd.to_numeric(raw_table['abs_mag_rough'], errors='coerce')
raw_table.dropna(subset=['gaiadr3_bp_rp', 'abs_mag_rough'], inplace=True)


diagonal_mask = (abs(raw_table['abs_mag_rough'] - (m * raw_table['gaiadr3_bp_rp'] + b)) <= tolerance)

filtered_table = raw_table[diagonal_mask]

#ADDING dScuti LABEL TO RELEVANT STARS

#label table
label_table = pd.read_csv("stats.csv",skiprows=[0],names=["TICID","Analysed","dScuti","Rpeak"])
label_table["dScuti"] = pd.to_numeric(label_table["dScuti"], errors="coerce").fillna(0).astype(int)

merged_table = pd.merge(filtered_table, label_table, on="TICID", how="left")

# --- Create mask for dScuti stars ---
dscuti_mask = merged_table["dScuti"] == 1

# --- Extract δ Scuti stars from filtered sample ---
filtered_dscuti_table = merged_table[dscuti_mask]

# Make a Python list of TIC IDs for confirmed δ Scuti stars
dscuti_tic_list = filtered_dscuti_table["TICID"].tolist()

#convert to integers
dscuti_tic_list = [int(tic) for tic in dscuti_tic_list]

print(dscuti_tic_list)
print(len(dscuti_tic_list))


[334177803, 333077513, 418824772, 168829745, 19924794, 71859994, 288404080, 420896293, 77390459, 155745933, 155772218, 410540533, 126006658, 70367797, 206144150, 413731775, 73806092, 214611227, 368759, 225709409, 381822792, 259365501, 288143236, 186287370, 49069582]
25


In [8]:
for id in dscuti_tic_list:
    pdf_creator(id)

/var/folders/pr/hmmmnv3x3jlf2f8nvv2vlkq40000gn/T/ipykernel_25925/1046325510.py:71: RuntimeWarning: invalid value encountered in sqrt
  amp = np.sqrt(sc) * fct * 1e6
/var/folders/pr/hmmmnv3x3jlf2f8nvv2vlkq40000gn/T/ipykernel_25925/1046325510.py:71: RuntimeWarning: invalid value encountered in sqrt
  amp = np.sqrt(sc) * fct * 1e6
/var/folders/pr/hmmmnv3x3jlf2f8nvv2vlkq40000gn/T/ipykernel_25925/1046325510.py:71: RuntimeWarning: invalid value encountered in sqrt
  amp = np.sqrt(sc) * fct * 1e6
/var/folders/pr/hmmmnv3x3jlf2f8nvv2vlkq40000gn/T/ipykernel_25925/1046325510.py:71: RuntimeWarning: invalid value encountered in sqrt
  amp = np.sqrt(sc) * fct * 1e6
